In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import vectfit

# Vectfit modification for non-conjugated complex poles

## Introduction
If the signals that are present in a system are *complex*, then the impulse responses in this system can also be complex. Correspondingly, the transfer function of this system can include complex poles that do not form complex conjugated pairs (i.e., frequency response becomes asymmetric with respect to zero frequency). For example, optical systems, such as laser interferometers, are often described as operators acting on complex amplitudes of the electromagnetic field. Many results of control theory remain applicable in the complex domain without any changes. However, some calculation methods need to be modified in order to work in the complex domain.

Unlike the original MATLAB package, `vectfit` for python can fit system transfer functions for complex-valued signals. Mathematically, the difference between such transfer functions and those that describe systems with real signals, is that the latter can only have complex poles that form conjugated pairs, whilst the former can have lone complex poles, or complex pole "pairs" of the form $s_p = \sigma_1+i (\omega_0\pm\omega_1)$ (i.e. are "symmetric" with respect to some axis $\Im s = \omega_0$ instead of $\Im s = 0$). 

This generalisation is based on the idea for `vectfit` modification described the following paper:
 * Spina, D., Ye, Y., Deschrijver, D., Bogaerts, W. and Dhaene, T. (2021), Complex vector fitting toolbox: a software package for the modelling and simulation of general linear and passive baseband systems. Electron. Lett., 57: 404-406.

In that paper, the authors remove the conjugateness constrain on from the original `vectfit` approach, described in
 * Gustavsen, B. (2009). Fast passivity enforcement for S-parameter models by perturbation of residue matrix eigenvalues. IEEE Transactions on advanced packaging, 33(1), 257-265
     
## How to use
The functionality for complex-valued systems is included in the main tool of the module -- `vectfit_auto()` function. Passing the argument `allow_nonconj=True` to this function enables the complex vector fitting algorithm. In this case, an initial distribution of single complex poles is generated instead of complex conjugate pairs. The iterative process uses functions in which the requirement for the poles being in complex conjugated pairs is removed according to the method suggested in `Spina, D. et al, 2021` (referenced above).

## Example

In [ ]:
# List of frequencies in Hz
freqs=np.logspace(-3,3,1000)
# List of arguments of the transfer function in the Laplace domain
s = 2*np.pi*freqs*1j

# Test transfer function 1, given by poles, residues, offset, and slope:
tst_poles = np.array([-1e-2+1e-2j,-1e-2-1e-2j,-1e-2+1j,-1e1,-5e1])
tst_residues = -tst_poles
tst_d=0
tst_h=0

# Evaluation of test function: list of complex values to fit
tst_tf = vectfit.model_polres(s, tst_poles, tst_residues, tst_d, tst_h)

# Vector fitting
# 1. standard vector fitting (bad result even for many parameters and lots of iterations
# because we have a lone complex pole)
fit1_poles, fit1_residues, fit1_d, fit1_h = \
    vectfit.vectfit_auto(tst_tf, s, n_complex_pairs=20, n_real_poles=2, n_iter=100) 
# 2. using allow_nonconj (gives a reasonably good fit with small number of initial poles)
fit2_poles, fit2_residues, fit2_d, fit2_h = \
    vectfit.vectfit_auto(tst_tf, s, \
                         allow_nonconj=True, \
                         n_complex_poles=3, \
                         n_real_poles=2 \
                        ) 

fit1_tf = vectfit.model_polres(s, fit1_poles, fit1_residues, fit1_d, fit1_h)
fit2_tf = vectfit.model_polres(s, fit2_poles, fit2_residues, fit2_d, fit2_h)

# Plotting results
plt.figure(figsize=(12,8))
plt.subplot(211)
plt.title("Vectfitting a function with an isolated complex pole")
plt.loglog(freqs, np.abs(tst_tf), label = "original function", color='black')
plt.loglog(freqs, np.abs(fit1_tf), label = "standard vectfit", ls=":", color='b')
plt.loglog(freqs, np.abs(fit2_tf), label = "modified vectfit (allow_nonconj=True)", ls="--", color='r')
plt.legend()
plt.ylabel("Amplitude")
plt.subplot(212)
plt.semilogx(freqs, np.angle(tst_tf)*180/np.pi, label = "original function", color='black')
plt.semilogx(freqs, np.angle(fit1_tf)*180/np.pi, label = "standard vectfit", ls=":", color='b')
plt.semilogx(freqs, np.angle(fit2_tf)*180/np.pi, label = "modified vectfit (allow_nonconj=True)", ls="--", color='r')
plt.legend()
plt.xlabel("Frequency, Hz")
plt.ylabel("Phase, degrees")
plt.show()
